In [2]:
import glob
import datasets

In [3]:
data = [f for f in glob.glob("results/*.json")]

In [4]:
len(data)

436779

In [5]:
data[0]

'results/2000461.json'

In [6]:
dataset = datasets.load_dataset(path="json", data_files=data)

Resolving data files:   0%|          | 0/436779 [00:00<?, ?it/s]

Using custom data configuration default-c572ac2f0a053e71


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/.cache/huggingface/datasets/json/default-c572ac2f0a053e71/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'date', 'content', 'summary'],
        num_rows: 436779
    })
})

In [18]:
import json
import re, os

regex_bracket = r"\(([^)]+)\)"
punctuation = '.,!?\'\[]();"'
unknown = set()

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def clean_article(article):
    article = cleanhtml(article)
    article = article.replace('\n', ' ')
    sentences = []
    words = []
    for word in article.split(' '):
        word = word.replace('–', '-')
        word = word.replace('__', '').replace('--', '')
        word = word.replace('&quot', '"')
        word = word.strip()
        if len(word) > 0:
            tokens = re.findall(r"[\w'\%\&\-\/\=\+\*$£]+|[\[\]().,!?\:;\"\“\”]", word)
            words += tokens
            try:
                if tokens[-1] in '.!?':
                    sentences.append(words)
                    words = []
            except:
                unknown.add(word)
    if words != []:
        if not words[-1][-1] in '.!?':
            words.append('.')
        sentences.append(words)
    return sentences

def get_string(sentences):
    all_sentence = []
    for sentence in sentences:
        all_sentence += sentence
    return ' '.join(all_sentence)


def process(batch):
    for i in range(len(batch['content'])):
        clean_data = {}
        article = batch['content'][i]
        summary = batch['summary'][i]
        if(len(article.split())>30 and len(summary.split())>10):
            article_arr = clean_article(article)
            summary_arr = clean_article(summary)
            clean_data['id'] = batch['id'][i]
            clean_data['url'] = batch['url'][i]

            article_v2 = get_string(article_arr).split()
            summary_v2 = get_string(summary_arr).split()

            if len(article_v2) < len(article.split()) or len(summary_v2) < len(summary.split()):
                print(str(batch['id'][i]))

            clean_data['clean_article'] = article_arr
            clean_data['clean_summary'] = summary_arr
            with open("liputan6_extend/"+str(clean_data['id'])+'.json', 'w') as json_file:
                json.dump(clean_data, json_file)

In [19]:
ds = dataset["train"].map(
    process,
    batched=True,
    batch_size=32,
    remove_columns=['id', 'url', 'title', 'date', 'content', 'summary']
)

  0%|          | 0/13650 [00:00<?, ?ba/s]

2016587
2084039
756737
576305
542849
2668643
2493496
2139709
2404262
2345180
2279227
2143619
2155070
2176187
2329590
2331950
2379166
2984692
711530


In [29]:
ds

Dataset({
    features: ['id', 'url', 'title', 'date', 'content', 'summary'],
    num_rows: 436779
})

In [24]:
dataset = datasets.load_dataset(path="json", data_files=[f for f in glob.glob("liputan6_extend/*.json")])

Resolving data files:   0%|          | 0/367693 [00:00<?, ?it/s]

Using custom data configuration default-9307b48c1d9e3dc4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/.cache/huggingface/datasets/json/default-9307b48c1d9e3dc4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
len([f for f in glob.glob("liputan6_extend/*.json")])

367693

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'clean_article', 'clean_summary'],
        num_rows: 367693
    })
})

In [28]:
def gabung(batch):
    return {
        'id': batch["id"],
        'url': batch["url"],
        'clean_article': [get_string(s) for s in batch['clean_article']],
        'clean_summary': [get_string(s) for s in batch['clean_summary']]
    }

In [30]:
ds = dataset.map(
    gabung,
    batched=True,
    batch_size=128,
    remove_columns=['id', 'url', 'clean_article', 'clean_summary'],
    num_proc=4
)

#0:   0%|          | 0/719 [00:00<?, ?ba/s]

#1:   0%|          | 0/719 [00:00<?, ?ba/s]

#2:   0%|          | 0/719 [00:00<?, ?ba/s]

#3:   0%|          | 0/719 [00:00<?, ?ba/s]

In [31]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'clean_article', 'clean_summary'],
        num_rows: 367693
    })
})

In [32]:
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel

df = pd.DataFrame(ds["train"][:1])
del df["id"]
del df["url"]
for column, typ in ds["train"].features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])
display(HTML(df.to_html()))

,clean_article,clean_summary
0,"Liputan6 . com , Jakarta - Kapolri Jendral Polisi Sutarman memerintahkan seluruh jajarannya agar mengawal dan mengamankan setiap tahapan Pemilu 2014 dengan baik . Sebab , setiap tahapan pemilu berpotensi menimbulkan kerawanan . Amanat itu disampaikan Sutarman saat apel operasi Mantap Brata 2014 untuk Pengamanan Pemilu di Lapangan Monumen Nasional , Jakarta , Kamis ( 30/1/2014 ) Ia menekankan kepada jajarannya untuk menjaga situasi kemanan kondusif serta memberikan rasa aman kepada peserta dan penyelenggara pemilu . "" Harus menjamin masyarakat menggunakan hak pilihnya dengan jernih sesuai hati nurani , "" imbau Sutarman . Selain itu , adanya intimidasi dari kelompok-kelompok tertentu yang kemungkinan terjadi di setiap wilayah saat pemilu juga harus dihentikan . Potensi tersebut merupakan tindak pidana pemilu . "" Terkait tindak pidana pemilu , kita bekerja sama dengan kejaksaan dan Bawaslu . Peran Gakkumdu juga akan dioptimalkan untuk menyelesaikan tuntas pelanggaran pemilu di seluruh tahapan , "" papar dia . Setiap pelanggaran yang dilaporkan masyarakat akan dinilai Kejaksaan , Sentra Gakkumdu , dan Polri . Sehingga polisi bisa menilai apakah pelangaran itu bentuknya bersifat administrasi , kode etik , atau tindak pidana . "" Setelah itu ditindaklanjuti sesuai pelanggaran yang terjadi , "" ungkap mantan Kabagreskrim itu . Ia menambahkan , saat ini logistik pemilu telah disiapkan . Semua pengamanan logistik merupakan tanggungjawab Polri . Mulai pendistribusian hingga ke TPS sampai kembalinya surat suara ke Kelompok Penyelenggara Pemungutan Suara . "" Karena itu perlu bersatu padu bersinergi dengan aparatur yang terlibat pengamanan pemilu . Sehingga dapat berjalan lancar aman jujur dan adil , "" tegas Sutarman . Karenanya , Sutarman meminta jajarannya terus melakukan latihan pengamanan di TPS , pengawalan logistik pemilu , pengamanan gedung-gedung penyelenggara pemilu yang bisa menjadi sasaran pelaku-pelaku untuk mengagalkan pemilu . "" Tingkatkan hingga latihan kontigensi , sehingga seluruh personel siap hadapi kontigensi , "" tukas Sutarman .",Kapolri Jendral Polisi Sutarman memerintahkan seluruh jajarannya agar mengawal dan mengamankan setiap tahapan Pemilu 2014 dengan baik .


In [33]:
import torch
import datasets
import statistics
import pandas as pd

from datetime import datetime
from functools import cached_property
from transformers import BertTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, EncoderDecoderModel
from config import *

class AbsSumExtract:
    
    @cached_property
    def tokenizer(self):
        tokenizer = BertTokenizer.from_pretrained(extract_model_checkpoint)
        tokenizer.bos_token = tokenizer.cls_token
        tokenizer.eos_token = tokenizer.sep_token
        return tokenizer

    @cached_property
    def extract_data(self):
        return ds["train"]
    
    @cached_property
    def model(self):
        model = EncoderDecoderModel.from_pretrained(extract_model_checkpoint)
        model.to("cuda:0")
        return model
    
    def generate_summary(self, batch):
        res_dict: dict = {
            "id": [],
            "summary": [],
            "generated_summary": []
        }
        inputs = self.tokenizer(batch["clean_article"], padding="max_length", truncation=True, max_length=encoder_max_length, return_tensors="pt")
        input_ids = inputs.input_ids.to("cuda:0")
        attention_mask = inputs.attention_mask.to("cuda:0")
        outputs = self.model.generate(
            input_ids,
            attention_mask=attention_mask,
            min_length=20,
            max_length=80, 
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
        )
        output_str = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        res_dict["id"] += batch["id"]
        res_dict["summary"] += batch["clean_summary"]
        res_dict["generated_summary"] += output_str
        
        last_id = res_dict["id"][-1]
        name = extract_model_checkpoint.split('/')[-1]
        fname = f"{extract_path}{name}-{last_id}-from_scrap-extraction_results.csv"
        
        df = pd.DataFrame(res_dict)
        df.to_csv(fname, index=False)
        
        return None
    
    def extract(self):
        _ = self.extract_data.map(
            self.generate_summary,
            batched=True,
            batch_size=extract_batch_size,
            remove_columns=["clean_article", "id"]
        )

In [34]:
extractcls = AbsSumExtract()

In [ ]:
extractcls.extract()

  0%|          | 0/11491 [00:00<?, ?ba/s]